In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
df = pd.read_csv('orig_facilities_faa.tsv', sep='\t', index_col=False, engine='python')
pd.set_option('display.max_columns', 103)
pd.set_option('display.max_rows', 110)
df.head()

,SiteNumber,Type,LocationID,EffectiveDate,Region,DistrictOffice,State,StateName,County,CountyState,City,FacilityName,Ownership,Use,Owner,OwnerAddress,OwnerCSZ,OwnerPhone,Manager,ManagerAddress,ManagerCSZ,ManagerPhone,ARPLatitude,ARPLatitudeS,ARPLongitude,ARPLongitudeS,ARPMethod,ARPElevation,ARPElevationMethod,MagneticVariation,MagneticVariationYear,TrafficPatternAltitude,ChartName,DistanceFromCBD,DirectionFromCBD,LandAreaCoveredByAirport,BoundaryARTCCID,BoundaryARTCCComputerID,BoundaryARTCCName,ResponsibleARTCCID,ResponsibleARTCCComputerID,ResponsibleARTCCName,TieInFSS,TieInFSSID,TieInFSSName,AirportToFSSPhoneNumber,TieInFSSTollFreeNumber,AlternateFSSID,AlternateFSSName,AlternateFSSTollFreeNumber,NOTAMFacilityID,NOTAMService,ActivationDate,AirportStatusCode,CertificationTypeDate,FederalAgreements,AirspaceDetermination,CustomsAirportOfEntry,CustomsLandingRights,MilitaryJointUse,MilitaryLandingRights,InspectionMethod,InspectionGroup,LastInspectionDate,LastOwnerInformationDate,FuelTypes,AirframeRepair,PowerPlantRepair,BottledOxygenType,BulkOxygenType,LightingSchedule,BeaconSchedule,ATCT,UNICOMFrequencies,CTAFFrequency,SegmentedCircle,BeaconColor,NonCommercialLandingFee,MedicalUse,SingleEngineGA,MultiEngineGA,JetEngineGA,HelicoptersGA,GlidersOperational,MilitaryOperational,Ultralights,OperationsCommercial,OperationsCommuter,OperationsAirTaxi,OperationsGALocal,OperationsGAItin,OperationsMilitary,OperationsDate,AirportPositionSource,AirportPositionSourceDate,AirportElevationSource,AirportElevationSourceDate,ContractFuelAvailable,TransientStorage,OtherServices,WindIndicator,IcaoIdentifier,MinimumOperationalNetwork
0,50009.*A,AIRPORT,'ADK,02/27/2020,AAL,NONE,AK,ALASKA,ALEUTIANS WEST,AK,ADAK ISLAND,ADAK,PU,PU,ALASKA DOT&PF SOUTHCOAST REGION,1500 ANTON LARSON,"KODIAK, AK 99615-2506",907-465-1784,INNOCENT DUSHKIN,P O BOX 1952,"ADAK, AK 99546",907-592-8026,51-53-00.8954N,186780.8954N,176-38-32.9277W,635912.9277W,E,19,S,07E,2000.0,NaN,W ALEUTIAN ISLS,0.0,W,NaN,ZAN,ZAN,ANCHORAGE,ZAP,ZAP,ANCHORAGE OCEANIC,N,CDB,COLD BAY,907-532-2466,1-800-478-7250,ENA,KENAI,1-866-864-1737,ADK,Y,04/01/1949,O,I B S 04/2005,N,NO OBJECTION,N,N,N,Y,F,F,7272019.0,1081991.0,A1,NONE,NONE,NONE,NONE,SEE RMK,SS-SR,N,NaN,122.9,Y,CG,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,104.0,NaN,0.0,0.0,100.0,28.0,06/30/2019,3RD PARTY SURVEY,2011-07-17 00:00:00.0,3RD PARTY SURVEY,2011-07-17 00:00:00.0,NaN,HGR,CARGO,Y,PADK,NaN
1,50016.1*A,AIRPORT,'AKK,02/27/2020,AAL,NONE,AK,ALASKA,KODIAK ISLAND,AK,AKHIOK,AKHIOK,PU,PU,ALASKA DOT&PF SOUTHCOAST REGION,PO BOX 112506,"JUNEAU, AK 99811-6900",907-269-0760,PHIL SMITH,1500 ANTON LARSON RD.,"KODIAK, AK 99615",907-487-4952,56-56-19.2870N,204979.2870N,154-10-57.2000W,555057.2000W,E,44,S,18E,2005.0,NaN,KODIAK,1.0,SSW,NaN,ZAN,ZAN,ANCHORAGE,ZAN,ZAN,ANCHORAGE,N,ENA,KENAI,907-283-7211,1-866-864-1737,NaN,NaN,NaN,AKK,Y,11/01/1979,O,NaN,NGY,NO OBJECTION,N,N,NaN,Y,C,C,7302019.0,NaN,NaN,NONE,NONE,NONE,NONE,NaN,NaN,N,122.8,122.8,N,NaN,N,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1500.0,NaN,100.0,NaN,12/31/2018,NACO,1997-03-18 00:00:00.0,NACO,1997-03-18 00:00:00.0,NaN,NaN,NaN,Y,PAKH,NaN
2,50017.*A,AIRPORT,'Z13,02/27/2020,AAL,NONE,AK,ALASKA,BETHEL,AK,AKIACHAK,AKIACHAK,PU,PU,ALASKA DOT&PF CENTRAL REGION,PO BOX 196900,"ANCHORAGE, AK 99519-6900",907-269-0770,LAWRENCE DAVIS,"STATE OF ALASKA DOT, 2016 AIRPORT ROAD","BETHEL, AK 99559",(907) 543-2498,60-54-49.7150N,219289.7150N,161-29-35.9850W,581375.9850W,E,22,S,19E,1985.0,NaN,MC GRATH,2.0,W,231.0,ZAN,ZAN,ANCHORAGE,ZAN,ZAN,ANCHORAGE,N,ENA,KENAI,907-283-7211,1-866-864-1737,NaN,NaN,NaN,ENA,Y,01/01/1968,O,NaN,N,NO OBJECTION,N,N,N,Y,C,C,6302017.0,NaN,NaN,NONE,NONE,NONE,NONE,SEE RMK,SEE RMK,N,NaN,122.9,Y,CG,N,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,3000.0,NaN,NaN,NaN,12/31/2016,STATE,2013-12-02 00:00:00.0,STATE,2013-12-02 00:00:00.0,NaN,NaN,CARGO,Y-L,NaN,NaN
3,50017.1*C,SEAPLANE BASE,'KKI,02/27/2020,AAL,NONE,AK,ALASKA,BETHEL,AK,AKIACHAK,AKIACHAK,PU,PU,PUBLIC DOMAIN,NaN,NaN,NaN,NONE,NaN,NaN,NaN,60-54-28.3130N,219268.3130N,161-26

In [3]:
df.shape

(19636, 103)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19636 entries, 0 to 19635
Columns: 103 entries, SiteNumber to MinimumOperationalNetwork
dtypes: float64(22), int64(1), object(80)
memory usage: 15.4+ MB


In [5]:
df.head()

,SiteNumber,Type,LocationID,EffectiveDate,Region,DistrictOffice,State,StateName,County,CountyState,City,FacilityName,Ownership,Use,Owner,OwnerAddress,OwnerCSZ,OwnerPhone,Manager,ManagerAddress,ManagerCSZ,ManagerPhone,ARPLatitude,ARPLatitudeS,ARPLongitude,ARPLongitudeS,ARPMethod,ARPElevation,ARPElevationMethod,MagneticVariation,MagneticVariationYear,TrafficPatternAltitude,ChartName,DistanceFromCBD,DirectionFromCBD,LandAreaCoveredByAirport,BoundaryARTCCID,BoundaryARTCCComputerID,BoundaryARTCCName,ResponsibleARTCCID,ResponsibleARTCCComputerID,ResponsibleARTCCName,TieInFSS,TieInFSSID,TieInFSSName,AirportToFSSPhoneNumber,TieInFSSTollFreeNumber,AlternateFSSID,AlternateFSSName,AlternateFSSTollFreeNumber,NOTAMFacilityID,NOTAMService,ActivationDate,AirportStatusCode,CertificationTypeDate,FederalAgreements,AirspaceDetermination,CustomsAirportOfEntry,CustomsLandingRights,MilitaryJointUse,MilitaryLandingRights,InspectionMethod,InspectionGroup,LastInspectionDate,LastOwnerInformationDate,FuelTypes,AirframeRepair,PowerPlantRepair,BottledOxygenType,BulkOxygenType,LightingSchedule,BeaconSchedule,ATCT,UNICOMFrequencies,CTAFFrequency,SegmentedCircle,BeaconColor,NonCommercialLandingFee,MedicalUse,SingleEngineGA,MultiEngineGA,JetEngineGA,HelicoptersGA,GlidersOperational,MilitaryOperational,Ultralights,OperationsCommercial,OperationsCommuter,OperationsAirTaxi,OperationsGALocal,OperationsGAItin,OperationsMilitary,OperationsDate,AirportPositionSource,AirportPositionSourceDate,AirportElevationSource,AirportElevationSourceDate,ContractFuelAvailable,TransientStorage,OtherServices,WindIndicator,IcaoIdentifier,MinimumOperationalNetwork
0,50009.*A,AIRPORT,'ADK,02/27/2020,AAL,NONE,AK,ALASKA,ALEUTIANS WEST,AK,ADAK ISLAND,ADAK,PU,PU,ALASKA DOT&PF SOUTHCOAST REGION,1500 ANTON LARSON,"KODIAK, AK 99615-2506",907-465-1784,INNOCENT DUSHKIN,P O BOX 1952,"ADAK, AK 99546",907-592-8026,51-53-00.8954N,186780.8954N,176-38-32.9277W,635912.9277W,E,19,S,07E,2000.0,NaN,W ALEUTIAN ISLS,0.0,W,NaN,ZAN,ZAN,ANCHORAGE,ZAP,ZAP,ANCHORAGE OCEANIC,N,CDB,COLD BAY,907-532-2466,1-800-478-7250,ENA,KENAI,1-866-864-1737,ADK,Y,04/01/1949,O,I B S 04/2005,N,NO OBJECTION,N,N,N,Y,F,F,7272019.0,1081991.0,A1,NONE,NONE,NONE,NONE,SEE RMK,SS-SR,N,NaN,122.9,Y,CG,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,104.0,NaN,0.0,0.0,100.0,28.0,06/30/2019,3RD PARTY SURVEY,2011-07-17 00:00:00.0,3RD PARTY SURVEY,2011-07-17 00:00:00.0,NaN,HGR,CARGO,Y,PADK,NaN
1,50016.1*A,AIRPORT,'AKK,02/27/2020,AAL,NONE,AK,ALASKA,KODIAK ISLAND,AK,AKHIOK,AKHIOK,PU,PU,ALASKA DOT&PF SOUTHCOAST REGION,PO BOX 112506,"JUNEAU, AK 99811-6900",907-269-0760,PHIL SMITH,1500 ANTON LARSON RD.,"KODIAK, AK 99615",907-487-4952,56-56-19.2870N,204979.2870N,154-10-57.2000W,555057.2000W,E,44,S,18E,2005.0,NaN,KODIAK,1.0,SSW,NaN,ZAN,ZAN,ANCHORAGE,ZAN,ZAN,ANCHORAGE,N,ENA,KENAI,907-283-7211,1-866-864-1737,NaN,NaN,NaN,AKK,Y,11/01/1979,O,NaN,NGY,NO OBJECTION,N,N,NaN,Y,C,C,7302019.0,NaN,NaN,NONE,NONE,NONE,NONE,NaN,NaN,N,122.8,122.8,N,NaN,N,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1500.0,NaN,100.0,NaN,12/31/2018,NACO,1997-03-18 00:00:00.0,NACO,1997-03-18 00:00:00.0,NaN,NaN,NaN,Y,PAKH,NaN
2,50017.*A,AIRPORT,'Z13,02/27/2020,AAL,NONE,AK,ALASKA,BETHEL,AK,AKIACHAK,AKIACHAK,PU,PU,ALASKA DOT&PF CENTRAL REGION,PO BOX 196900,"ANCHORAGE, AK 99519-6900",907-269-0770,LAWRENCE DAVIS,"STATE OF ALASKA DOT, 2016 AIRPORT ROAD","BETHEL, AK 99559",(907) 543-2498,60-54-49.7150N,219289.7150N,161-29-35.9850W,581375.9850W,E,22,S,19E,1985.0,NaN,MC GRATH,2.0,W,231.0,ZAN,ZAN,ANCHORAGE,ZAN,ZAN,ANCHORAGE,N,ENA,KENAI,907-283-7211,1-866-864-1737,NaN,NaN,NaN,ENA,Y,01/01/1968,O,NaN,N,NO OBJECTION,N,N,N,Y,C,C,6302017.0,NaN,NaN,NONE,NONE,NONE,NONE,SEE RMK,SEE RMK,N,NaN,122.9,Y,CG,N,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,3000.0,NaN,NaN,NaN,12/31/2016,STATE,2013-12-02 00:00:00.0,STATE,2013-12-02 00:00:00.0,NaN,NaN,CARGO,Y-L,NaN,NaN
3,50017.1*C,SEAPLANE BASE,'KKI,02/27/2020,AAL,NONE,AK,ALASKA,BETHEL,AK,AKIACHAK,AKIACHAK,PU,PU,PUBLIC DOMAIN,NaN,NaN,NaN,NONE,NaN,NaN,NaN,60-54-28.3130N,219268.3130N,161-26

In [6]:
cols_to_drop = ['DistrictOffice', 'CountyState', 'ARPLatitude', 'ARPLongitude', 'ARPMethod', 'ARPElevationMethod', 'LandAreaCoveredByAirport', 'BoundaryARTCCComputerID',
                'ResponsibleARTCCComputerID', 'TieInFSS', 'AirportToFSSPhoneNumber', 'AlternateFSSID', 'AlternateFSSName',
                'AlternateFSSTollFreeNumber', 'ActivationDate', 'CertificationTypeDate', 'FederalAgreements','AirspaceDetermination',
                'InspectionMethod', 'InspectionGroup', 'LastInspectionDate', 'LastOwnerInformationDate', 'SegmentedCircle', 'AirportPositionSource',
                'AirportPositionSourceDate', 'AirportElevationSource', 'AirportElevationSourceDate', 'ContractFuelAvailable', 'TransientStorage',
                'OtherServices', 'WindIndicator', 'MinimumOperationalNetwork']

df.drop(columns=cols_to_drop, inplace=True)

In [7]:
df.columns = [re.sub(r'(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|(?<=[A-Z])(?=ID)', '_', x) 
              for x in df.columns]

In [8]:
df.columns = df.columns.str.strip().str.lower()

In [9]:
df.columns = [re.sub(r'(arp_)|(tie_in_)|(_s$)', '', x) for x in df.columns]

In [10]:
df.head(1)

,site_number,type,location_id,effective_date,region,state,state_name,county,city,facility_name,ownership,use,owner,owner_address,owner_csz,owner_phone,manager,manager_address,manager_csz,manager_phone,latitude,longitude,elevation,magnetic_variation,magnetic_variation_year,traffic_pattern_altitude,chart_name,distance_from_cbd,direction_from_cbd,boundary_artcc_id,boundary_artcc_name,responsible_artcc_id,responsible_artcc_name,fss_id,fss_name,fss_toll_free_number,notam_facility_id,notam_service,airport_status_code,customs_airport_of_entry,customs_landing_rights,military_joint_use,military_landing_rights,fuel_types,airframe_repair,power_plant_repair,bottled_oxygen_type,bulk_oxygen_type,lighting_schedule,beacon_schedule,atct,unicom_frequencies,ctaf_frequency,beacon_color,non_commercial_landing_fee,medical_use,single_engine_ga,multi_engine_ga,jet_engine_ga,helicopters_ga,gliders_operational,military_operational,ultralights,operations_commercial,operations_commuter,operations_air_taxi,operations_ga_local,operations_ga_itin,operations_military,operations_date,icao_identifier
0,50009.*A,AIRPORT,'ADK,02/27/2020,AAL,AK,ALASKA,ALEUTIANS WEST,ADAK ISLAND,ADAK,PU,PU,ALASKA DOT&PF SOUTHCOAST REGION,1500 ANTON LARSON,"KODIAK, AK 99615-2506",907-465-1784,INNOCENT DUSHKIN,P O BOX 1952,"ADAK, AK 99546",907-592-8026,186780.8954N,635912.9277W,19,07E,2000.0,NaN,W ALEUTIAN ISLS,0.0,W,ZAN,ANCHORAGE,ZAP,ANCHORAGE OCEANIC,CDB,COLD BAY,1-800-478-7250,ADK,Y,O,N,N,N,Y,A1,NONE,NONE,NONE,NONE,SEE RMK,SS-SR,N,NaN,122.9,CG,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,104.0,NaN,0.0,0.0,100.0,28.0,06/30/2019,PADK


In [11]:
df[['latitude', 'longitude']] = df[['latitude', 'longitude']].applymap(
    lambda x: float(x.strip('SW')) / -3600 if x.endswith('W') or x.endswith('S')
    else float(x.strip('NE')) / 3600)

In [12]:
df['location_id'] = df['location_id'].apply(lambda x: x[1:])

In [13]:
df['effective_date'] =  pd.to_datetime(df['effective_date'])  

In [14]:
df.head()

,site_number,type,location_id,effective_date,region,state,state_name,county,city,facility_name,ownership,use,owner,owner_address,owner_csz,owner_phone,manager,manager_address,manager_csz,manager_phone,latitude,longitude,elevation,magnetic_variation,magnetic_variation_year,traffic_pattern_altitude,chart_name,distance_from_cbd,direction_from_cbd,boundary_artcc_id,boundary_artcc_name,responsible_artcc_id,responsible_artcc_name,fss_id,fss_name,fss_toll_free_number,notam_facility_id,notam_service,airport_status_code,customs_airport_of_entry,customs_landing_rights,military_joint_use,military_landing_rights,fuel_types,airframe_repair,power_plant_repair,bottled_oxygen_type,bulk_oxygen_type,lighting_schedule,beacon_schedule,atct,unicom_frequencies,ctaf_frequency,beacon_color,non_commercial_landing_fee,medical_use,single_engine_ga,multi_engine_ga,jet_engine_ga,helicopters_ga,gliders_operational,military_operational,ultralights,operations_commercial,operations_commuter,operations_air_taxi,operations_ga_local,operations_ga_itin,operations_military,operations_date,icao_identifier
0,50009.*A,AIRPORT,ADK,2020-02-27,AAL,AK,ALASKA,ALEUTIANS WEST,ADAK ISLAND,ADAK,PU,PU,ALASKA DOT&PF SOUTHCOAST REGION,1500 ANTON LARSON,"KODIAK, AK 99615-2506",907-465-1784,INNOCENT DUSHKIN,P O BOX 1952,"ADAK, AK 99546",907-592-8026,51.883582,-176.642480,19,07E,2000.0,NaN,W ALEUTIAN ISLS,0.0,W,ZAN,ANCHORAGE,ZAP,ANCHORAGE OCEANIC,CDB,COLD BAY,1-800-478-7250,ADK,Y,O,N,N,N,Y,A1,NONE,NONE,NONE,NONE,SEE RMK,SS-SR,N,NaN,122.9,CG,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,104.0,NaN,0.0,0.0,100.0,28.0,06/30/2019,PADK
1,50016.1*A,AIRPORT,AKK,2020-02-27,AAL,AK,ALASKA,KODIAK ISLAND,AKHIOK,AKHIOK,PU,PU,ALASKA DOT&PF SOUTHCOAST REGION,PO BOX 112506,"JUNEAU, AK 99811-6900",907-269-0760,PHIL SMITH,1500 ANTON LARSON RD.,"KODIAK, AK 99615",907-487-4952,56.938691,-154.182556,44,18E,2005.0,NaN,KODIAK,1.0,SSW,ZAN,ANCHORAGE,ZAN,ANCHORAGE,ENA,KENAI,1-866-864-1737,AKK,Y,O,N,N,NaN,Y,NaN,NONE,NONE,NONE,NONE,NaN,NaN,N,122.8,122.8,NaN,N,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1500.0,NaN,100.0,NaN,12/31/2018,PAKH
2,50017.*A,AIRPORT,Z13,2020-02-27,AAL,AK,ALASKA,BETHEL,AKIACHAK,AKIACHAK,PU,PU,ALASKA DOT&PF CENTRAL REGION,PO BOX 196900,"ANCHORAGE, AK 99519-6900",907-269-0770,LAWRENCE DAVIS,"STATE OF ALASKA DOT, 2016 AIRPORT ROAD","BETHEL, AK 99559",(907) 543-2498,60.913810,-161.493329,22,19E,1985.0,NaN,MC GRATH,2.0,W,ZAN,ANCHORAGE,ZAN,ANCHORAGE,ENA,KENAI,1-866-864-1737,ENA,Y,O,N,N,N,Y,NaN,NONE,NONE,NONE,NONE,SEE RMK,SEE RMK,N,NaN,122.9,CG,N,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,3000.0,NaN,NaN,NaN,12/31/2016,NaN
3,50017.1*C,SEAPLANE BASE,KKI,2020-02-27,AAL,AK,ALASKA,BETHEL,AKIACHAK,AKIACHAK,PU,PU,PUBLIC DOMAIN,NaN,NaN,NaN,NONE,NaN,NaN,NaN,60.907865,-161.435077,18,19E,1985.0,NaN,MC GRATH,0.0,S,ZAN,ANCHORAGE,ZAN,ANCHORAGE,ENA,KENAI,1-866-864-1737,ENA,Y,O,N,N,N,N,NaN,NONE,NONE,NONE,NONE,NaN,NaN,N,NaN,122.9,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,12/31/2016,NaN
4,50020.*A,AIRPORT,AKI,2020-02-27,AAL,AK,ALASKA,BETHEL,AKIAK,AKIAK,PU,PU,ALASKA DOT&PF CENTRAL REGION,PO BOX 196900,"ANCHORAGE, AK 99519-6900",907-269-0760,LAWERENCE DAVIS,P O BOX 505,"BETHEL, AK 99559",(907) 543-2498,60.902897,-161.230583,39,16E,2005.0,NaN,MC GRATH,0.0,SW,ZAN,ANCHORAGE,ZAN,ANCHORAGE,ENA,KENAI,1-866-864-1737,ENA,Y,O,N,N,N,Y,NaN,NONE,NONE,NONE,NONE,SEE RMK,SS-SR,N,NaN,122.9,CG,N,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1000.0,NaN,NaN,NaN,12/31/2016,PFAK


In [15]:
bool_cols = ['notam_service', 'customs_airport_of_entry', 'customs_landing_rights', 'military_joint_use', 'military_landing_rights',
             'atct', 'non_commercial_landing_fee', 'medical_use']

In [16]:
for col in bool_cols:
    df[col] = df[col].map({'Y': True, 'N': False})
#df[[bool_cols]].map({'Y': True, 'N': False})

In [17]:
df.head()

,site_number,type,location_id,effective_date,region,state,state_name,county,city,facility_name,ownership,use,owner,owner_address,owner_csz,owner_phone,manager,manager_address,manager_csz,manager_phone,latitude,longitude,elevation,magnetic_variation,magnetic_variation_year,traffic_pattern_altitude,chart_name,distance_from_cbd,direction_from_cbd,boundary_artcc_id,boundary_artcc_name,responsible_artcc_id,responsible_artcc_name,fss_id,fss_name,fss_toll_free_number,notam_facility_id,notam_service,airport_status_code,customs_airport_of_entry,customs_landing_rights,military_joint_use,military_landing_rights,fuel_types,airframe_repair,power_plant_repair,bottled_oxygen_type,bulk_oxygen_type,lighting_schedule,beacon_schedule,atct,unicom_frequencies,ctaf_frequency,beacon_color,non_commercial_landing_fee,medical_use,single_engine_ga,multi_engine_ga,jet_engine_ga,helicopters_ga,gliders_operational,military_operational,ultralights,operations_commercial,operations_commuter,operations_air_taxi,operations_ga_local,operations_ga_itin,operations_military,operations_date,icao_identifier
0,50009.*A,AIRPORT,ADK,2020-02-27,AAL,AK,ALASKA,ALEUTIANS WEST,ADAK ISLAND,ADAK,PU,PU,ALASKA DOT&PF SOUTHCOAST REGION,1500 ANTON LARSON,"KODIAK, AK 99615-2506",907-465-1784,INNOCENT DUSHKIN,P O BOX 1952,"ADAK, AK 99546",907-592-8026,51.883582,-176.642480,19,07E,2000.0,NaN,W ALEUTIAN ISLS,0.0,W,ZAN,ANCHORAGE,ZAP,ANCHORAGE OCEANIC,CDB,COLD BAY,1-800-478-7250,ADK,True,O,False,False,False,True,A1,NONE,NONE,NONE,NONE,SEE RMK,SS-SR,False,NaN,122.9,CG,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,104.0,NaN,0.0,0.0,100.0,28.0,06/30/2019,PADK
1,50016.1*A,AIRPORT,AKK,2020-02-27,AAL,AK,ALASKA,KODIAK ISLAND,AKHIOK,AKHIOK,PU,PU,ALASKA DOT&PF SOUTHCOAST REGION,PO BOX 112506,"JUNEAU, AK 99811-6900",907-269-0760,PHIL SMITH,1500 ANTON LARSON RD.,"KODIAK, AK 99615",907-487-4952,56.938691,-154.182556,44,18E,2005.0,NaN,KODIAK,1.0,SSW,ZAN,ANCHORAGE,ZAN,ANCHORAGE,ENA,KENAI,1-866-864-1737,AKK,True,O,False,False,NaN,True,NaN,NONE,NONE,NONE,NONE,NaN,NaN,False,122.8,122.8,NaN,False,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1500.0,NaN,100.0,NaN,12/31/2018,PAKH
2,50017.*A,AIRPORT,Z13,2020-02-27,AAL,AK,ALASKA,BETHEL,AKIACHAK,AKIACHAK,PU,PU,ALASKA DOT&PF CENTRAL REGION,PO BOX 196900,"ANCHORAGE, AK 99519-6900",907-269-0770,LAWRENCE DAVIS,"STATE OF ALASKA DOT, 2016 AIRPORT ROAD","BETHEL, AK 99559",(907) 543-2498,60.913810,-161.493329,22,19E,1985.0,NaN,MC GRATH,2.0,W,ZAN,ANCHORAGE,ZAN,ANCHORAGE,ENA,KENAI,1-866-864-1737,ENA,True,O,False,False,False,True,NaN,NONE,NONE,NONE,NONE,SEE RMK,SEE RMK,False,NaN,122.9,CG,False,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,3000.0,NaN,NaN,NaN,12/31/2016,NaN
3,50017.1*C,SEAPLANE BASE,KKI,2020-02-27,AAL,AK,ALASKA,BETHEL,AKIACHAK,AKIACHAK,PU,PU,PUBLIC DOMAIN,NaN,NaN,NaN,NONE,NaN,NaN,NaN,60.907865,-161.435077,18,19E,1985.0,NaN,MC GRATH,0.0,S,ZAN,ANCHORAGE,ZAN,ANCHORAGE,ENA,KENAI,1-866-864-1737,ENA,True,O,False,False,False,False,NaN,NONE,NONE,NONE,NONE,NaN,NaN,False,NaN,122.9,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,12/31/2016,NaN
4,50020.*A,AIRPORT,AKI,2020-02-27,AAL,AK,ALASKA,BETHEL,AKIAK,AKIAK,PU,PU,ALASKA DOT&PF CENTRAL REGION,PO BOX 196900,"ANCHORAGE, AK 99519-6900",907-269-0760,LAWERENCE DAVIS,P O BOX 505,"BETHEL, AK 99559",(907) 543-2498,60.902897,-161.230583,39,16E,2005.0,NaN,MC GRATH,0.0,SW,ZAN,ANCHORAGE,ZAN,ANCHORAGE,ENA,KENAI,1-866-864-1737,ENA,True,O,False,False,False,True,NaN,NONE,NONE,NONE,NONE,SEE RMK,SS-SR,False,NaN,122.9,CG,False,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1000.0,NaN,NaN,NaN,12/31/2016,PFAK


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19636 entries, 0 to 19635
Data columns (total 71 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   site_number                 19636 non-null  object        
 1   type                        19636 non-null  object        
 2   location_id                 19636 non-null  object        
 3   effective_date              19636 non-null  datetime64[ns]
 4   region                      19636 non-null  object        
 5   state                       19636 non-null  object        
 6   state_name                  19636 non-null  object        
 7   county                      19636 non-null  object        
 8   city                        19636 non-null  object        
 9   facility_name               19636 non-null  object        
 10  ownership                   19636 non-null  object        
 11  use                         19636 non-null  object    

In [19]:
na_value_dict = {'traffic_pattern_altitude': 1000, 'notam_service': False, 'customs_airport_of_entry': False, 'customs_landing_rights': False, 'military_joint_use': False, 'military_landing_rights': False,
             'atct': False, 'non_commercial_landing_fee': False, 'medical_use': False}

In [20]:
df.fillna(value=na_value_dict, inplace=True)

In [21]:
df.head()

,site_number,type,location_id,effective_date,region,state,state_name,county,city,facility_name,ownership,use,owner,owner_address,owner_csz,owner_phone,manager,manager_address,manager_csz,manager_phone,latitude,longitude,elevation,magnetic_variation,magnetic_variation_year,traffic_pattern_altitude,chart_name,distance_from_cbd,direction_from_cbd,boundary_artcc_id,boundary_artcc_name,responsible_artcc_id,responsible_artcc_name,fss_id,fss_name,fss_toll_free_number,notam_facility_id,notam_service,airport_status_code,customs_airport_of_entry,customs_landing_rights,military_joint_use,military_landing_rights,fuel_types,airframe_repair,power_plant_repair,bottled_oxygen_type,bulk_oxygen_type,lighting_schedule,beacon_schedule,atct,unicom_frequencies,ctaf_frequency,beacon_color,non_commercial_landing_fee,medical_use,single_engine_ga,multi_engine_ga,jet_engine_ga,helicopters_ga,gliders_operational,military_operational,ultralights,operations_commercial,operations_commuter,operations_air_taxi,operations_ga_local,operations_ga_itin,operations_military,operations_date,icao_identifier
0,50009.*A,AIRPORT,ADK,2020-02-27,AAL,AK,ALASKA,ALEUTIANS WEST,ADAK ISLAND,ADAK,PU,PU,ALASKA DOT&PF SOUTHCOAST REGION,1500 ANTON LARSON,"KODIAK, AK 99615-2506",907-465-1784,INNOCENT DUSHKIN,P O BOX 1952,"ADAK, AK 99546",907-592-8026,51.883582,-176.642480,19,07E,2000.0,1000.0,W ALEUTIAN ISLS,0.0,W,ZAN,ANCHORAGE,ZAP,ANCHORAGE OCEANIC,CDB,COLD BAY,1-800-478-7250,ADK,True,O,False,False,False,True,A1,NONE,NONE,NONE,NONE,SEE RMK,SS-SR,False,NaN,122.9,CG,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,104.0,NaN,0.0,0.0,100.0,28.0,06/30/2019,PADK
1,50016.1*A,AIRPORT,AKK,2020-02-27,AAL,AK,ALASKA,KODIAK ISLAND,AKHIOK,AKHIOK,PU,PU,ALASKA DOT&PF SOUTHCOAST REGION,PO BOX 112506,"JUNEAU, AK 99811-6900",907-269-0760,PHIL SMITH,1500 ANTON LARSON RD.,"KODIAK, AK 99615",907-487-4952,56.938691,-154.182556,44,18E,2005.0,1000.0,KODIAK,1.0,SSW,ZAN,ANCHORAGE,ZAN,ANCHORAGE,ENA,KENAI,1-866-864-1737,AKK,True,O,False,False,False,True,NaN,NONE,NONE,NONE,NONE,NaN,NaN,False,122.8,122.8,NaN,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1500.0,NaN,100.0,NaN,12/31/2018,PAKH
2,50017.*A,AIRPORT,Z13,2020-02-27,AAL,AK,ALASKA,BETHEL,AKIACHAK,AKIACHAK,PU,PU,ALASKA DOT&PF CENTRAL REGION,PO BOX 196900,"ANCHORAGE, AK 99519-6900",907-269-0770,LAWRENCE DAVIS,"STATE OF ALASKA DOT, 2016 AIRPORT ROAD","BETHEL, AK 99559",(907) 543-2498,60.913810,-161.493329,22,19E,1985.0,1000.0,MC GRATH,2.0,W,ZAN,ANCHORAGE,ZAN,ANCHORAGE,ENA,KENAI,1-866-864-1737,ENA,True,O,False,False,False,True,NaN,NONE,NONE,NONE,NONE,SEE RMK,SEE RMK,False,NaN,122.9,CG,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,3000.0,NaN,NaN,NaN,12/31/2016,NaN
3,50017.1*C,SEAPLANE BASE,KKI,2020-02-27,AAL,AK,ALASKA,BETHEL,AKIACHAK,AKIACHAK,PU,PU,PUBLIC DOMAIN,NaN,NaN,NaN,NONE,NaN,NaN,NaN,60.907865,-161.435077,18,19E,1985.0,1000.0,MC GRATH,0.0,S,ZAN,ANCHORAGE,ZAN,ANCHORAGE,ENA,KENAI,1-866-864-1737,ENA,True,O,False,False,False,False,NaN,NONE,NONE,NONE,NONE,NaN,NaN,False,NaN,122.9,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,12/31/2016,NaN
4,50020.*A,AIRPORT,AKI,2020-02-27,AAL,AK,ALASKA,BETHEL,AKIAK,AKIAK,PU,PU,ALASKA DOT&PF CENTRAL REGION,PO BOX 196900,"ANCHORAGE, AK 99519-6900",907-269-0760,LAWERENCE DAVIS,P O BOX 505,"BETHEL, AK 99559",(907) 543-2498,60.902897,-161.230583,39,16E,2005.0,1000.0,MC GRATH,0.0,SW,ZAN,ANCHORAGE,ZAN,ANCHORAGE,ENA,KENAI,1-866-864-1737,ENA,True,O,False,False,False,True,NaN,NONE,NONE,NONE,NONE,SEE RMK,SS-SR,False,NaN,122.9,CG,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1000.0,NaN,NaN,NaN,12/31/2016,PFAK


In [22]:
df.to_csv('../Clean-Data/facilities_clean.tsv', sep='\t')